# S&P500


https://medium.com/mlearning-ai/predict-sp500-stock-price-with-python-machine-learning-sentiment-analysis-a296dc276353

In [71]:
import pandas as pd
import re

In [72]:
df = pd.read_csv('Dataset/analyst_ratings_processed.csv')

In [73]:
df.head(10)

,Unnamed: 0,title,date,stock
0,0.0,Stocks That Hit 52-Week Highs On Friday,2020-06-05 10:30:00-04:00,A
1,1.0,Stocks That Hit 52-Week Highs On Wednesday,2020-06-03 10:45:00-04:00,A
2,2.0,71 Biggest Movers From Friday,2020-05-26 04:30:00-04:00,A
3,3.0,46 Stocks Moving In Friday's Mid-Day Session,2020-05-22 12:45:00-04:00,A
4,4.0,B of A Securities Maintains Neutral on Agilent...,2020-05-22 11:38:00-04:00,A
5,5.0,"CFRA Maintains Hold on Agilent Technologies, L...",2020-05-22 11:23:00-04:00,A
6,6.0,"UBS Maintains Neutral on Agilent Technologies,...",2020-05-22 09:36:00-04:00,A
7,7.0,Agilent Technologies shares are trading higher...,2020-05-22 09:07:00-04:00,A
8,8.0,Wells Fargo Maintains Overweight on Agilent Te...,2020-05-22 08:37:00-04:00,A
9,9.0,10 Biggest Price Target Changes For Friday,2020-05-22 08:06:00-04:00,A


In [74]:
df.shape

(1400469, 4)

In [75]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1400469 entries, 0 to 1400468
Data columns (total 4 columns):
 #   Column      Non-Null Count    Dtype  
---  ------      --------------    -----  
 0   Unnamed: 0  1399180 non-null  float64
 1   title       1400469 non-null  object 
 2   date        1399180 non-null  object 
 3   stock       1397891 non-null  object 
dtypes: float64(1), object(3)
memory usage: 42.7+ MB


In [76]:
data = df.drop(columns='Unnamed: 0')

In [77]:
data.head()

,title,date,stock
0,Stocks That Hit 52-Week Highs On Friday,2020-06-05 10:30:00-04:00,A
1,Stocks That Hit 52-Week Highs On Wednesday,2020-06-03 10:45:00-04:00,A
2,71 Biggest Movers From Friday,2020-05-26 04:30:00-04:00,A
3,46 Stocks Moving In Friday's Mid-Day Session,2020-05-22 12:45:00-04:00,A
4,B of A Securities Maintains Neutral on Agilent...,2020-05-22 11:38:00-04:00,A


In [78]:
data.isnull().sum().sort_values(ascending=False)/len(data)*100

stock    0.184081
date     0.092041
title    0.000000
dtype: float64

In [79]:
data[data['date'].isna()]

,title,date,stock
2298,Sterne Agee Provides Color on Aaron's,NaN,NaN
3568,Goldman Sachs Updates on Car Parts,NaN,NaN
10257,American Campus to Offer 15M Shares,NaN,NaN
10287,American Campus Communities Reports Q2 FFOM of...,NaN,NaN
10941,Positive Seeking Alpha Article on Aceto Corpor...,NaN,NaN
...,...,...,...
1397623,Zynga Loses Its CTO of Mobile -AllThingsD,NaN,NaN
1397757,Looks Like Zynga Just Bought OMGPOP For $200 M...,NaN,NaN
1397782,Zynga Lead Game Designer Leaving the Company -...,NaN,NaN
1399443,Waiting on Several IPOs This Morning:,NaN,NaN


In [80]:
data = data.dropna()

In [81]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1397891 entries, 0 to 1400468
Data columns (total 3 columns):
 #   Column  Non-Null Count    Dtype 
---  ------  --------------    ----- 
 0   title   1397891 non-null  object
 1   date    1397891 non-null  object
 2   stock   1397891 non-null  object
dtypes: object(3)
memory usage: 42.7+ MB


In [82]:
data.tail()

,title,date,stock
1400464,Top Narrow Based Indexes For August 29,2011-08-29 10:41:00-04:00,ZX
1400465,Recap: Wednesday's Top Percentage Gainers and ...,2011-06-22 16:44:00-04:00,ZX
1400466,UPDATE: Oppenheimer Color on China Zenix Auto ...,2011-06-21 08:26:00-04:00,ZX
1400467,Oppenheimer Initiates China Zenix At Outperfor...,2011-06-21 05:59:00-04:00,ZX
1400468,China Zenix Auto International Opens For Tradi...,2011-05-12 09:36:00-04:00,ZX


In [83]:
data['date'] = data['date'].map(lambda x: str(x)[:-6])

In [84]:
data['date'] = pd.to_datetime(data['date']).map(lambda x: x.date())

In [85]:
data.tail()

,title,date,stock
1400464,Top Narrow Based Indexes For August 29,2011-08-29,ZX
1400465,Recap: Wednesday's Top Percentage Gainers and ...,2011-06-22,ZX
1400466,UPDATE: Oppenheimer Color on China Zenix Auto ...,2011-06-21,ZX
1400467,Oppenheimer Initiates China Zenix At Outperfor...,2011-06-21,ZX
1400468,China Zenix Auto International Opens For Tradi...,2011-05-12,ZX


In [86]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1397891 entries, 0 to 1400468
Data columns (total 3 columns):
 #   Column  Non-Null Count    Dtype 
---  ------  --------------    ----- 
 0   title   1397891 non-null  object
 1   date    1397891 non-null  object
 2   stock   1397891 non-null  object
dtypes: object(3)
memory usage: 42.7+ MB


In [87]:
s = data['date']>=pd.to_datetime("2018-01-01").date()

s

0           True
1           True
2           True
3           True
4           True
           ...  
1400464    False
1400465    False
1400466    False
1400467    False
1400468    False
Name: date, Length: 1397891, dtype: bool

In [88]:
data = data.drop(data[data['date']<pd.to_datetime("2018-01-01").date()].index)

In [89]:
data.tail()

,title,date,stock
1400403,China Zenix Auto Shares Halted News Pending,2018-06-13,ZX
1400404,"China Zenix Auto Q1 EPS $0.08, Made $130.123M ...",2018-05-17,ZX
1400405,"China Zenix Auto Reports Q4 EPS $0.03, Sales $...",2018-03-15,ZX
1400406,UPDATE: Not Seeing News In Chinese Momentum St...,2018-01-05,ZX
1400407,Chinese Nano-Cap Momentum Stocks Sharply Highe...,2018-01-05,ZX


In [90]:
data.shape

(402168, 3)

In [91]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 402168 entries, 0 to 1400407
Data columns (total 3 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   title   402168 non-null  object
 1   date    402168 non-null  object
 2   stock   402168 non-null  object
dtypes: object(3)
memory usage: 12.3+ MB


In [94]:
data['stock'].nunique()

4554

: 